# Web Scraping con Python

In [1]:
from selenium import webdriver

In [2]:
from bs4 import BeautifulSoup as bs

In [3]:
import pandas as pd

In [4]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin

## SCRAPEAR MI DIARIO DE PELICULAS

Ok, ahora configura webdriver para usar Chrome por default, localiza tu chromedriver y usalo en este comando

In [5]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

Arma 3 listas, una para metros cuadrados, otra para precio y otra para ubicacion

In [6]:
titulosucio = []
añosucio = []
postersucio = []

In [23]:
titulos = []
precios = []
stocks = []

Y arma un driver.get para especificar la página de donde vamos a trabajar

In [32]:
driver.get("https://books.toscrape.com/")

In [7]:
driver.get("https://web-de-peliculas-92543.web.app/")

## Extracción de Datos

Guarda el código HTML de la página en un objeto Contenido

In [9]:
contenido = driver.page_source

Conviertelo a BeautifulSoup

In [10]:
soup = bs(contenido)

Arma un for en el que vayas agregando a nuestras listas los datos necesarios con a.find, usa . append para ir agregando los datos a las listas

In [13]:
# soup.find_all("article", attrs = {"class": "product_pod"}) para la pagina de libros

In [11]:
estructura_pelis = soup.find_all("div", attrs = {"class": "movie-card"})

Arma un for para limpiar precio

In [12]:
for pelicula in estructura_pelis: 
    titulo = pelicula.find("div", attrs = {"class": "movie-title"})
    titulosucio.append(titulo.text)

    año = pelicula.find("div", attrs = {"class": "movie-year"})
    añosucio.append(año.text)

    poster = pelicula.find("img", attrs = {"class": "movie-poster"})
    if poster and poster.get("src"):
        postersucio.append(poster["src"])  
    else:
        postersucio.append(None)  

Arma un for para limpiar ubicación

Arma un for para limpiar m2

## Almacenamiento

Guarda tus datos en un dataframe

In [15]:
data = {"Nombre": titulosucio, 
        "Año": añosucio,
        "Poster": postersucio}

In [17]:
df = pd.DataFrame(data)
df

,Nombre,Año,Poster
0,La piel que habitó,2011,https://m.media-amazon.com/images/M/MV5BYjgzMD...
1,Nadie duerme esta noche 2,2021,https://m.media-amazon.com/images/M/MV5BZjg2Mj...
2,The Woman,2011,https://m.media-amazon.com/images/M/MV5BZWM3YT...
3,El despertar de los muertos,2004,https://m.media-amazon.com/images/M/MV5BNzNjZG...
4,Saint Seiya: Soul of Gold,2015,https://images.justwatch.com/poster/310238737/...
...,...,...,...
762,La maldición de Halloween,1995,https://m.media-amazon.com/images/M/MV5BY2YyN2...
763,Cabin Fever 3: Patient Zero,2014,https://m.media-amazon.com/images/M/MV5BMTQxMT...
764,Scream 3,2000,https://m.media-amazon.com/images/M/MV5BMzc4MT...
765,Ejército de Frankenstein,2013,https://m.media-amazon.com/images/M/MV5BMjA1ND...


Exporta tu Dataframe a un archivo CSV y abrelo en Excel

## SCRAPEAR UN APAGINA DE LIBROS PARA SCRAPING

In [30]:
# Configuramos el driver de chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

titulos = []
precios = []
stocks = []

url = "http://books.toscrape.com"

while True:
    titulos_sucios = []
    precios_sucios = []
    stocks_sucio = []

    # urlaux = urljoin(url,f"/catalogue/page-{page}.html")

    driver.get(url)

    # Extracción de datos
    contenido = driver.page_source

    soup = bs(contenido)

    articles = soup.find_all("article",attrs={"class":"product_pod"})

    for libro in articles:
        titulo_paso1 = libro.find("h3").find("a") #La etiqueta que normalmente no podemos buscar, esta dentro de "a"
        titulo = titulo_paso1["title"]
        titulos_sucios.append(titulo)
        #titulos_sucios.append(titulo.text)

        precio = libro.find("p",attrs={"class":"price_color"})
        precios_sucios.append(precio.text)

        stock = libro.find("p",attrs={"class":"instock availability"})
        stocks_sucio.append(stock.text)

    # Limpiando los datos
    for libro in titulos_sucios:
        titulos.append(libro.replace("\n","").strip())

    for libro in precios_sucios:
        precios.append(libro.replace("\n","").strip())

    for libro in stocks_sucio:
        stocks.append(libro.replace("\n","").strip())

    #Encuentra la siguiente página que scrapear
    siguientepag = soup.select_one("li.next>a")
    if siguientepag:
        siguienteurl = siguientepag.get("href") #href es un término de html que se refiere a link, o sea, el botón de siguientepag a donde me va a llevar
        url = urljoin(url, siguienteurl) #Unir el url que ya tenía, con el siguiente url
    else:
        break

In [31]:
# Almacenamiento
df = pd.DataFrame({"titulo":titulos,"precio":precios,"stock":stocks})
df

,titulo,precio,stock
0,A Light in the Attic,£51.77,In stock
1,Tipping the Velvet,£53.74,In stock
2,Soumission,£50.10,In stock
3,Sharp Objects,£47.82,In stock
4,Sapiens: A Brief History of Humankind,£54.23,In stock
...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),£16.97,In stock
998,1st to Die (Women's Murder Club #1),£53.98,In stock


In [35]:
len(precios)

1000